In [ ]:
#QUESTION 1 - Automated S3 bucket cleanup using AWS Lambda and Boto3

import boto3
from datetime import datetime, timedelta, timezone
import pytz

utc = pytz.UTC

challenge.datetime_start = utc.localize(challenge.datetime_start)

challenge.datetime_end = utc.localize(challenge.datetime_end)

def lambda_handler(event, context):
    
    # Define the S3 bucket and region
    bucket_name = 'serverlesstestin'
    region_name = 'ap-south-1'

    # Initialize Boto3 S3 client
    s3 = boto3.client('s3', region_name=region_name)

    # Get the current time
    current_time = datetime.now()

    # Calculate the timestamp for one day ago
    one_day_ago = current_time - timedelta(days=1)

    try:
        # List objects in the S3 bucket
        response = s3.list_objects_v2(Bucket=bucket_name)

        # Check if objects are present in the bucket
        if 'Contents' in response:
            # Iterate through each object
            for obj in response['Contents']:
                # Get the object key and last modified time
                object_key = obj['Key']
                last_modified_time = obj['LastModified']

                # Check if the object is older than a day
                if last_modified_time < one_day_ago:
                    # Delete the object
                    s3.delete_object(Bucket=bucket_name, Key=object_key)
                    print(f"Deleted object: {object_key}")

        return {
            'statusCode': 200,
            'body': 'Files older than a day have been deleted successfully.'
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': f'Error: {str(e)}'
        }


In [ ]:
# QUESTION 2 - Automated instance management using AWS lambda and boto3

import boto3

def lambda_handler(event, context):
    # Create an EC2 client
    ec2_client = boto3.client('ec2')

    # Specify the tag key and value to identify instances for stopping and starting
    stop_tag_key = 'Auto-Stop'
    stop_tag_value = 'Auto-Stop'
    start_tag_key = 'Auto-Start'
    start_tag_value = 'Auto-Start'

    # Retrieve instances based on the specified stop tag key-value pair and stop them
    stop_instances(ec2_client, stop_tag_key, stop_tag_value)

    # Retrieve instances based on the specified start tag key-value pair and start them
    start_instances(ec2_client, start_tag_key, start_tag_value)

    return {
        'statusCode': 200,
        'body': 'EC2 instances managed based on tag key-value pairs'
    }

def get_instances_by_tag(ec2_client, tag_key, tag_value):

    instances = []

    # Describe instances with the specified tag
    response = ec2_client.describe_instances(
        Filters=[
            {'Name': f'tag:{tag_key}', 'Values': [tag_value]}
        ]
    )

    # Extract instance IDs from the response
    for reservation in response['Reservations']:
        for instance in reservation['Instances']:
            instances.append(instance['InstanceId'])

    return instances

def stop_instances(ec2_client, tag_key, tag_value):

    instances = get_instances_by_tag(ec2_client, tag_key, tag_value)
    if instances:
        ec2_client.stop_instances(InstanceIds=instances)
        print(f"Stopped instances: {', '.join(instances)}")

def start_instances(ec2_client, tag_key, tag_value):

    instances = get_instances_by_tag(ec2_client, tag_key, tag_value)
    if instances:
        ec2_client.start_instances(InstanceIds=instances)
        print(f"Started instances: {', '.join(instances)}")


In [1]:
#QUESTION 3 - Monitor unencrypted S3 buckets using AWS Lambda and Boto3

import boto3

def lambda_handler(event, context):
    # Create an S3 client
    s3_client = boto3.client('s3')

    # List all S3 buckets
    response = s3_client.list_buckets()

    # Initialize a dictionary to store bucket names and their encryption status
    bucket_encryption_status = {}

    # Check encryption status for each bucket
    for bucket_info in response['Buckets']:
        bucket_name = bucket_info['Name']
        encryption_status = check_bucket_encryption(s3_client, bucket_name)
        bucket_encryption_status[bucket_name] = encryption_status

    # Print the encryption status of each bucket
    for bucket_name, encryption_status in bucket_encryption_status.items():
        print(f"Bucket: {bucket_name}, Encrypted: {encryption_status}")

    return {
        'statusCode': 200,
        'body': 'S3 bucket encryption status checked'
    }

def check_bucket_encryption(s3_client, bucket_name):
    try:
        response = s3_client.get_bucket_encryption(Bucket=bucket_name)
        return True
    except s3_client.exceptions.ClientError as e:
        if e.response['Error']['Code'] == 'ServerSideEncryptionConfigurationNotFoundError':
            return False
        else:
            raise


In [ ]:
# QUESTION 4 - Monitor and Alert High AWS Billing Using AWS Lambda, Boto3, and SNS

import boto3
import datetime

def lambda_handler(event, context):
    # Initialize boto3 clients for CloudWatch and SNS
    cloudwatch_client = boto3.client('cloudwatch')
    sns_client = boto3.client('sns')

    # Retrieve the AWS billing metric from CloudWatch
    metric_name = 'EstimatedCharges'
    namespace = 'AWS/Billing'
    period = 86400  # 24 hours (1 day)
    statistics = 'Maximum'

    response = cloudwatch_client.get_metric_statistics(
        Namespace=namespace,
        MetricName=metric_name,
        StartTime=get_start_time(period),
        EndTime=get_end_time(),
        Period=period,
        Statistics=[statistics]
    )

    # Check if there are data points returned
    if 'Datapoints' in response:
        datapoints = response['Datapoints']
        if datapoints:
            # Sort datapoints by timestamp in descending order
            sorted_datapoints = sorted(datapoints, key=lambda x: x['Timestamp'], reverse=True)
            latest_datapoint = sorted_datapoints[0]
            billing_amount = latest_datapoint['Maximum']

            # Compare the billing amount with a threshold
            threshold = 50.0  # Threshold for billing amount
            if billing_amount > threshold:
                # Send an SNS notification
                topic_arn = 'arn:aws:sns:us-east-1:541717357993:billing'
                message = f"AWS billing exceeds threshold: ${billing_amount}"
                sns_client.publish(TopicArn=topic_arn, Message=message)
                print("SNS notification sent.")
            else:
                print("Billing is within threshold.")
        else:
            print("No data points found for the billing metric.")
    else:
        print("No data points found for the billing metric.")


def get_start_time(period):
    # Calculate start time based on the period
    return datetime.datetime.utcnow() - datetime.timedelta(seconds=2 * period)


def get_end_time():
    # End time is the current time
    return datetime.datetime.utcnow()
